In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
import math
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from scipy.cluster.hierarchy import dendrogram, linkage

init_notebook_mode(connected=True)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**Table Tennis Exploratory Data Analysis**

In [ ]:
olympics = pd.read_csv('../input/athlete_events.csv')

In [ ]:
# Clean the Team column
# Some of the entries in the column contain the country name followed by a dash and a number, such as China-2.
new_Team = [team[:-2] if team[-2] == '-' else team for team in olympics.Team]
olympics.Team = new_Team

tt_men = olympics.loc[(olympics.Sport == "Table Tennis") & (olympics.Sex == 'M')]
tt_women = olympics.loc[(olympics.Sport == "Table Tennis") & (olympics.Sex == 'F')]
tt_men_singles = olympics.loc[(olympics.Event == "Table Tennis Men's Singles")]

In [ ]:
# Histograms for the age, height, and weight of table tennis players, grouped by gender
tt_men.Age.plot(kind='hist', title = 'Athlete Age Distribution', alpha=0.5, label = 'Male')
tt_women.Age.plot(kind='hist', alpha=0.5, label = 'Female')
plt.xlabel('Age')
plt.legend(loc ='upper right')
plt.show()

tt_men.Height.plot(kind='hist', title = 'Athlete Height Distribution', alpha=0.5, label = 'Male')
tt_women.Height.plot(kind='hist', alpha=0.5, label = 'Female')
plt.xlabel('Height (cm)')
plt.legend(loc ='upper right')
plt.show()

tt_men.Weight.plot(kind='hist', title = 'Athlete Weight Distribution', alpha=0.5, label = 'Male')
tt_women.Weight.plot(kind='hist', alpha=0.5, label = 'Female')
plt.xlabel('Weight (kg)')
plt.legend(loc ='upper right')
plt.show()

In [ ]:
# Examine the huge outlier in the weight histogram
tt_women.loc[tt_women.Weight == tt_women.Weight.max()]

In [ ]:
# Looking for a particular player I knew
tt_women.loc[tt_women.Name == 'Ariel Yenhua Hsing']

In [ ]:
# Ariel defeated the oldest female athlete, Ni Xia Lian, in the round of 64
tt_women.loc[tt_women.Age == tt_women.Age.max()]

In [ ]:
# Is the oldest athlete male or female?
tt_men.loc[tt_men.Age == tt_men.Age.max()]

In [ ]:
men_singles_bronze = tt_men_singles.loc[tt_men_singles.Medal == 'Bronze'].groupby('Year')['Year'].count()
men_singles_bronze

# Upon further research, there was no third place game played in 1992, leading to two bronze medals.

In [ ]:
total_medals_country_men = tt_men.loc[pd.notnull(tt_men.Medal)].groupby('Team').Team.count()
total_medals_country_men

In [ ]:
total_medals_country_women = tt_women.loc[pd.notnull(tt_women.Medal)].groupby('Team').Team.count()
total_medals_country_women

In [ ]:
df = pd.DataFrame(data = {'Medals':total_medals_country_men}, index=total_medals_country_men.index).reset_index()
print(df)

In [ ]:
trace = go.Choropleth(
            locations = df['Team'],
            locationmode='country names',
            z = df['Medals'],
            text = df['Team'],
            autocolorscale = False,
            reversescale = True,
            colorscale = 'Viridis',
            marker = dict(
                line = dict(
                    color = 'rgb(0,0,0)',
                    width = 1)
            ),
            colorbar = dict(
                title = 'Medals',
                tickprefix = '')
        )

data = [trace]
layout = go.Layout(
    title = 'Table Tennis Medals by Country (Men)',
    geo = dict(
        showframe = True,
        showlakes = False,
        showcoastlines = True,
        projection = dict(
            type = 'natural earth'
        )
    )
)

fig = dict( data=data, layout=layout )
iplot(fig)

In [ ]:
df2 = pd.DataFrame(data = {'Medals':total_medals_country_women}, index=total_medals_country_women.index).reset_index()
print(df2)

In [ ]:
trace2 = go.Choropleth(
            locations = df2['Team'],
            locationmode='country names',
            z = df2['Medals'],
            text = df2['Team'],
            autocolorscale = False,
            reversescale = True,
            colorscale = 'Viridis',
            marker = dict(
                line = dict(
                    color = 'rgb(0,0,0)',
                    width = 1)
            ),
            colorbar = dict(
                title = 'Medals',
                tickprefix = '')
        )

data2 = [trace2]
layout = go.Layout(
    title = 'Table Tennis Medals by Country (Women)',
    geo = dict(
        showframe = True,
        showlakes = False,
        showcoastlines = True,
        projection = dict(
            type = 'natural earth'
        )
    )
)

fig2 = dict( data=data2, layout=layout )
iplot(fig2)

**Swimming Event 100 Meters**

In [ ]:
df_events = pd.read_csv("../input/athlete_events.csv")
df_events = df_events.loc[df_events.Year >= 1994]
df_regions = pd.read_csv("../input/noc_regions.csv")
df_joined = df_regions.merge(df_events, on='NOC')

The first thing I noticed is that the there are two files that aren't very large so I am going to just read the whole files in. If the files were larger I wouldn't attempt to read the whole files in.

In [ ]:
# df_joined[df_joined.Sport.unique()]
tmp = df_joined.loc[df_joined.Sport == 'Swimming', ['Event', 'Height', 'Weight']]
swimming_events = pd.Series(tmp.Event.unique()[1:10])
# print()

plt.figure()
sns.stripplot(x='Event', y='Height', data=tmp.loc[tmp.Event.isin(swimming_events)])
plt.xticks(rotation=60)
plt.show()
plt.figure()
sns.stripplot(x='Event', y='Weight', data=tmp.loc[tmp.Event.isin(swimming_events)])
plt.xticks(rotation=60)
plt.show()

In [ ]:
missing_vals = df_joined.isna().sum(axis=0).reset_index()
missing_vals.columns = ['Label', 'Count']
missing_vals = missing_vals.loc[missing_vals.Count > 0]
missing_vals

In [ ]:
plt.figure()
barplot = sns.barplot(x='Label', y='Count', data=missing_vals)
barplot.set(title='Count of Missing Values', xlabel='variables', ylabel='missing values')
plt.xticks(rotation=60)
plt.show()

In [ ]:
# which country has the most medals?
print(df_events.isna().sum(axis=0).reset_index())
print(df_regions.isna().sum(axis=0).reset_index())

In [ ]:
swimming_events = df_joined.loc[df_joined.Sport == 'Swimming']
swimming_events = swimming_events.Event.unique()
event_subset = df_joined.Event.isin(swimming_events[11:20])
event_subset = df_joined.loc[event_subset]

In [ ]:
plt.figure()
sns.stripplot(x='Event', y='Height', data=event_subset)
plt.xticks(rotation=90)
plt.show()

In [ ]:
swimming_events = df_joined.loc[df_joined.Sport == 'Swimming']
swimming_events_100m_subset = pd.Series(data=["Swimming Women's 100 metres Backstroke",
                        "Swimming Men's 100 metres Freestyle",
                        "Swimming Women's 100 metres Butterfly",
                        "Swimming Men's 100 metres Breaststroke",
                        "Swimming Women's 100 metres Freestyle",
                        "Swimming Men's 100 metres Backstroke",
                        "Swimming Men's 100 metres Butterfly",
                        "Swimming Women's 100 metres Breaststroke",
                        "Swimming Men's 100 Yard Backstroke",
                        "Swimming Men's 100 yard Freestyle"])
swimming_events_100m = swimming_events.loc[swimming_events.Event.isin(swimming_events_100m_subset)]

In [ ]:
plt.figure()
sns.stripplot(x='Event', y='Height', hue='Sex', data=swimming_events_100m)
plt.xticks(rotation=90)
plt.show()

plt.figure()
sns.stripplot(x='Event', y='Weight', hue='Sex', data=swimming_events_100m)
plt.xticks(rotation=90)
plt.show()

In [ ]:
swimming_events_100m.loc['Placed'] = swimming_events_100m.Medal.apply(lambda x: 1 if pd.isnull(x) else 0)
se100m_winners = swimming_events_100m.loc[~swimming_events_100m.Medal.isnull()]

In [ ]:
plt.figure()
# swimming_events_100m.groupby('NOC').Placed.sum()
sns.countplot('NOC', data = se100m_winners)
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure()
# swimming_events_100m.groupby('NOC').Placed.sum()
sns.countplot('NOC', data = se100m_winners.loc[~se100m_winners.NOC.isin(['USA','AUS'])])
plt.xticks(rotation=90)
plt.show()

In [ ]:
# plt.figure()
# swimming_events_100m.groupby('NOC').Placed.sum()
# sns.stripplot('NOC', data = se100m_winners.loc[~se100m_winners.NOC.isin(['USA','AUS'])],
#              hue="event")
# plt.xticks(rotation=90)
# plt.show()
# se100m_winners.loc[~se100m_winners.NOC.isin(['USA','AUS']), ['Year', 'Event', 'NOC', 'region']].sort_values(by='Year')
plt.figure()
sns.countplot('Team', data = se100m_winners.loc[~se100m_winners.NOC.isin(['USA','AUS']), ['Year', 'Event', 'Team', 'region']].sort_values(by='Year'))
plt.xticks(rotation=90)
plt.show()

# se100m_winners.loc[se100m_winners.NOC == 'NED', ['Year', 'Event']].sort_values('Year')
# se100m_winners.loc[se100m_winners.Year.isin([2012,2016]), ['Year', 'Event', 'region']].sort_values(by=['Year', 'Event'])
plt.figure()
sns.countplot('Team', data = se100m_winners.loc[se100m_winners.Year.isin([2012,2016]), ['Year', 'NOC', 'Event', 'Team']])
plt.xticks(rotation=90)
plt.show()

In [ ]:
# se100m_winners.head(5)
tmp = se100m_winners.groupby(['Year','Team']).count().Medal.reset_index()
# tmp.Year = pd.to_datetime(tmp.Year, yearfirst=True)
# print(tmp)
champs = tmp.loc[(tmp.Medal > 5) | (tmp.Medal == 0)].Team.unique()
tmp = tmp.loc[~tmp.Team.isin(champs)]

plt.figure()
plt.subplots(figsize=(20,10))
sns.lineplot(x="Year", y="Medal", hue = 'Team',
             data=tmp)
plt.xticks(np.linspace(1994, 2016, num=12))
plt.show()

**Track and Field: 100M Sprint**

**Figure Skating**



In [ ]:
data = pd.read_csv('../input/athlete_events.csv')
noc = pd.read_csv('../input/noc_regions.csv')

Figure skating has been an Olympic sport since the 1908 Summer Olympics. There are a total of 5 events within the category, a solo event for men and women, and 3 paired events. Originally it started as Men's Singles, Women's Singles, and Mixed Pairs, but in 1976 Ice Dancing became an event and in 2014 a Mixed Team event was also created. In this exploratory data analysis we will focus on the single events.

In [ ]:
print(data[data['Sport'] == 'Figure Skating'].groupby('Event').Event.count())

In [ ]:
print(data[data['Sport'] == 'Figure Skating'].groupby('Event').Event.count())

In [ ]:
print(data[data['Event'] == "Figure Skating Men's Special Figures"])

In [ ]:
print(data[data['Event'] == 'Figure Skating Mixed Pairs'].groupby(['Year','Team']).ID.count().head(10))

We see that the data contains two athlete ID's per team for mixed pairs event.

In [ ]:
print(data.groupby('Sport').Age.mean().sort_values(ascending=True).head())

In [ ]:
mfs = data[data['Event'] == "Figure Skating Men's Singles"]
ffs = data[data['Event'] == "Figure Skating Women's Singles"]

In [ ]:
print(mfs.shape,"\n", ffs.shape)

In [ ]:
## For Men
print(mfs.mean(), '\n')
print(mfs.median())

In [ ]:
## For Women
print(ffs.mean(), '\n')
print(ffs.median())

In [ ]:
sns.distplot(mfs.Age.dropna(), color = 'blue', hist = False, label='men')
sns.distplot(ffs.Age.dropna(), color = 'red', hist = False, label='women')
plt.ylabel('Density')
plt.title('Ages in Figure Skating')
plt.show()

Pretty hard cutoff by 30 for women, and most women appear to be under twenty years old. Men's age is more normally distributed, with some athletes that compete over 30.

In [ ]:
sns.distplot(mfs.Height.dropna(), color = 'blue', hist=False, label = 'men')
sns.distplot(ffs.Height.dropna(), color = 'red', hist=False, label = 'women')
plt.ylabel('Density')
plt.title('Height(in cm) in Figure Skating')
plt.show()

Nothing particularly exciting about this graph. Both genders have normally distributed heights.

In [ ]:
sns.distplot(mfs.Weight.dropna(), color = 'blue', hist=False, label = 'men')
sns.distplot(ffs.Weight.dropna(), color = 'red', hist=False, label = 'women')
plt.ylabel('Density')
plt.title('Weight(in kg) in Figure Skating')
plt.show()

More normal curves. Are figure skaters potentially underweight for their respective heights/age? One way to test this is with BMI (not the most reliable methods, but possible given our data).

In [ ]:
tempm = mfs[['Weight', 'Height']].dropna()
tempf = ffs[['Weight', 'Height']].dropna()
tempm.Height = tempm.Height/100.0
tempf.Height = tempf.Height/100.0
BMIm = tempm.Weight/(tempm.Height*tempm.Height)
BMIf = tempf.Weight/(tempf.Height*tempf.Height)

In [ ]:
sns.distplot(BMIm, color = 'blue', hist=False, label = 'men')
sns.distplot(BMIf, color = 'red', hist=False, label= 'women')
plt.ylabel('Density')
plt.title('BMI Scores in Figure Skating')
plt.show()


A "healthy" BMI value is considered to be between 18.5 to 24. We can see here that the men mostly fit into that area, but female figure skaters have a lower BMI overall. Could these lower BMI values be associated with what country these athletes are coming from?

In [ ]:
tempbmi = ffs.loc[BMIf.index,['Name','Team', 'Age', 'NOC', 'Year', 'Medal']]
tempbmi['BMI'] = BMIf
tempbmi = pd.merge(tempbmi, noc, how='left', on ='NOC')

In [ ]:
print(tempbmi.groupby('region').BMI.mean().sort_values(ascending=True).head(10), "\n")
print(tempbmi.groupby('region').BMI.mean().sort_values(ascending=True).tail(10), "\n")

We see that while some countries have good BMI score means, others have a low mean value.

**Medals Won in Figure Skating**

In [ ]:
mfs = pd.merge(mfs, noc, how='left', on='NOC')
ffs = pd.merge(ffs, noc, how='left', on='NOC')

**Medals per NOC Committee**

In [ ]:

## Total medals for men's singles by country
menmedals = mfs.groupby('region').Medal.count().sort_values(ascending=False)
print(menmedals.head(10), "\n\n\n")

## Total medels for women's singles by country
womenmedals = ffs.groupby('region').Medal.count().sort_values(ascending=False)
print(womenmedals.head(10))

We see some similar countries, such as the USA, Russia, Austria, and Germany that are top competitors for both the men's and women's events.

In [ ]:
## Medals won per Rank per Country
print(mfs.groupby(['region', 'Medal']).Medal.count().sort_values(ascending = False).head(15), "\n\n\n\n")
print(ffs.groupby(['region', 'Medal']).Medal.count().sort_values(ascending = False).head(15))

In [ ]:
print(mfs.groupby('Medal').region.nunique(), '\n\n\n')
print(ffs.groupby('Medal').region.nunique())

In [ ]:
print(mfs.groupby('region').Medal.count().sort_values(ascending=False).head(10), '\n\n\n')
print(ffs.groupby('region').Medal.count().sort_values(ascending=False).head(10))

In [ ]:
## Number of NOC committees participating per event
print(ffs.region.nunique())
print(mfs.region.nunique(), '\n')

## Number of Medals obtained from 1908 to 2014 per event
print(mfs.Medal.count())
print(ffs.Medal.count())

We see that out of 44 countries that compete, very few of them have ever achieved medalists (less than half of them receive any medals at all). For female athletes in particular, the USA alone won almost one third of all possible medals in the singles event for the entire lifetime of the event at the Olympics.

In [ ]:
## The number of medals won per athlete
m_med = mfs.groupby(['Name', 'Team']).Medal.count()
f_med = ffs.groupby(['Name', 'Team']).Medal.count()
print(m_med.sort_values(ascending=False).head(10), '\n\n\n')
print(f_med.sort_values(ascending=False).head(10))

The highest number of Medals won by an Olympic Figure Skater was 4. We see that it is not too uncommon for figure skaters to attempt multiple years if they were successful previously.

In [ ]:
## The number of Olympics participated in per athlete
m_oly = mfs.groupby(['Name', 'Team']).Year.nunique()
f_oly = ffs.groupby(['Name', 'Team']).Year.nunique()
print(m_oly.sort_values(ascending=False).head(10), '\n\n\n')
print(f_oly.sort_values(ascending=False).head(10))

It will probably be more interesting to observe the proportion between their medal count and Olympic attempts (their "success rate" at the Olympic Games).

In [ ]:
m_attempt = pd.merge(pd.DataFrame(m_med).reset_index(), pd.DataFrame(m_oly).reset_index(), on = 'Name')
f_attempt = pd.merge(pd.DataFrame(f_med).reset_index(), pd.DataFrame(f_oly).reset_index(), on = 'Name')
m_attempt['Prop'] = m_attempt.Medal/m_attempt.Year
f_attempt['Prop'] = f_attempt.Medal/f_attempt.Year

In [ ]:
print(m_attempt.loc[:,['Name','Team_x','Medal','Prop']].sort_values(['Prop','Medal'], ascending = False).head(10))

In [ ]:
print(f_attempt.loc[:,['Name','Team_x','Medal','Prop']].sort_values(['Prop','Medal'], ascending = False).head(10))

We see that Gillis Emanuel Grafström was the only athlete in figure skating to win 4 medals over 4 Olympic Games. However, there were several strong athletes that came back to the Olympics to win a second medal. We do see some problems though. Aleksey Yevgenyevich Urmanov is listed twice for males, which seems odd. We also see something wrong for Katarina Witt in East Germany. So we need investigate further.

In [ ]:
print(data[data['Name'] == 'Aleksey Yevgenyevich Urmanov'].loc[:,['Name','Team','NOC','Year','Medal']], '\n\n\n')

In [ ]:
print(data[data['Name'] == 'Katarina Witt'].loc[:,['Name','Team','NOC','Year', 'Medal']], '\n\n\n')

Because Katarina Witt participiated in Olymipcs from 1984 and 1994, her Team changed from East Germany to Germany after the reunification of Germany in 1989. From the actual data, we see she participated 3 times, and won 2 medals.

Participants

In [ ]:
m = pd.DataFrame(mfs.groupby('Year').Name.count()).reset_index()
f = pd.DataFrame(ffs.groupby('Year').Name.count()).reset_index()

#plt.scatter(mfs.Year.unique(), mfs.groupby('Year').Name.count(), color = 'blue')
#plt.scatter(pd.DataFrame(ffs.groupby('Year').Name.count()).reindex())
plt.plot(m['Year'], m['Name'], color = 'blue', label = 'men')
plt.plot(f['Year'], f['Name'], color = 'red', label = 'women')
plt.xlabel('Year')
plt.ylabel('# of Participants')
plt.title('# of Participants per Year')
plt.legend(loc = 'lower right')
plt.show()


We see that the number of competitors in the events shifted drastically, and only stabilized close to above 25 competitors after the year 2000.

In [ ]:
m_ppy = pd.DataFrame(mfs.groupby(['Year','region']).Name.nunique()).reset_index()
f_ppy = pd.DataFrame(ffs.groupby(['Year','region']).Name.nunique()).reset_index()

print(m_ppy[m_ppy['Year'] == 2014], '\n\n\n')
print(f_ppy[f_ppy['Year'] == 2014])

For the 2014 Winter Olympics, Canada, Japan, and the USA brought a lot of members for both events.



In [ ]:
m_ppy = m_ppy[m_ppy['Year'] >= 2000]
f_ppy = f_ppy[f_ppy['Year'] >= 2000]


In [ ]:
sns.set_palette('deep', 44)
plt.stackplot(m_ppy.pivot('Year','region','Name').index, np.transpose(m_ppy.pivot('Year','region', 'Name').fillna(0)))
l1 = plt.legend(bbox_to_anchor=(1.38, 0), labels = m_ppy.pivot('Year','region','Name').columns[0:20], labelspacing = -2)
l2 = plt.legend(bbox_to_anchor=(1.67,0.5), labels = m_ppy.pivot('Year', 'region', 'Name').columns[20:33], labelspacing = -2)
plt.gca().add_artist(l1)
plt.xlabel('Year')
plt.xticks([2002, 2006, 2010, 2014])
plt.ylabel('Participants')
plt.title('Stackplot of Participants per Region')
plt.show()

We see that in the four most recent Winter Olympics, countries like the USA, Japan, Canada, and France have been able to send multiple athletes for the men's single event. We see other countries such as North Korea, Poland, and Slovenia that were able to participate only once or twice, with only a single athlete representing their country.

In [ ]:

plt.stackplot(f_ppy.pivot('Year','region','Name').index, np.transpose(f_ppy.pivot('Year','region', 'Name').fillna(0)))
l1 = plt.legend(bbox_to_anchor=(1.38, 0), labels = f_ppy.pivot('Year','region','Name').columns[0:20], labelspacing = -2)
l2 = plt.legend(bbox_to_anchor=(1.67,0.5), labels = f_ppy.pivot('Year', 'region', 'Name').columns[20:35], labelspacing = -2)
plt.gca().add_artist(l1)
plt.xlabel('Year')
plt.xticks([2002, 2006, 2010, 2014])
plt.ylabel('Participants')
plt.title('Stackplot of Participants per Region')
plt.show()

For women, we see USA, Russia, Canada, and Japan that are able to send more than one athlete per Olympics. However, unlike the male teams, we see other countries such as China, Uzbekistan, and Australia that are more consistently able to compete in this event.

**Model**

Sport

In [ ]:
olympics = pd.read_csv('../input/athlete_events.csv')

In [ ]:
olympics = olympics.loc[olympics.Year >= 1994]

# Clean the Team column
# Some of the entries in the column contain the country name followed by a dash and a number, such as China-2.
pd.options.mode.chained_assignment = None
new_Team = [team[:-2] if team[-2] == '-' else team for team in olympics.Team]
olympics.Team = new_Team

top_countries = olympics.groupby('Team').Team.count().sort_values(ascending = False)[0:40,]
olympics = olympics.loc[olympics.Team.isin(top_countries.index)]

olympics_men = olympics.loc[olympics.Sex == 'M']
olympics_women = olympics.loc[olympics.Sex == 'F']

In [ ]:
# Determine the most popular event within each sport category for men and women
df_men = olympics_men.groupby(['Sport','Event']).Sport.count()
sports_men = df_men.index.get_level_values(0)
events_men = df_men.loc[df_men.groupby(sports_men).idxmax()].index.get_level_values(level=1).tolist()

df_women = olympics_women.groupby(['Sport','Event']).Sport.count()
sports_women = df_women.index.get_level_values(0)
events_women = df_women.loc[df_women.groupby(sports_women).idxmax()].index.get_level_values(level=1).tolist()

In [ ]:
olympics_men = olympics_men.loc[olympics_men.Event.isin(events_men)]
olympics_women = olympics_women.loc[olympics_women.Event.isin(events_women)]

In [ ]:
medals_men = olympics_men.loc[pd.notnull(olympics_men.Medal)].groupby(['Team','Event']).Team.count().unstack(level=0).fillna(0)
medals_women = olympics_women.loc[pd.notnull(olympics_women.Medal)].groupby(['Team','Event']).Team.count().unstack(level=0).fillna(0)

In [ ]:

mergings_men = linkage(medals_men, method = 'ward')
plt.figure(figsize=(10,20))
dendrogram(mergings_men, labels = medals_men.index, leaf_rotation = 0, leaf_font_size=15, orientation='right')
plt.show()

# This dendrogram shows how closely certain men's events from different sports are related,
# based on how countries perform in those events. For example:
# Judo is close to taekwondo
# Table tennis is close to badminton
# Freestyle skiing is close to figure skating and snowboarding, etc.

In [ ]:
mergings_women = linkage(medals_women, method = 'ward')
plt.figure(figsize=(10,20))
dendrogram(mergings_women, labels = medals_women.index, leaf_rotation = 0, leaf_font_size=15, orientation='right')
plt.show()

In [ ]:
Country

In [ ]:
olympics = olympics.loc[olympics.Year >= 1994]

# Clean the Team column
# Some of the entries in the column contain the country name followed by a dash and a number, such as China-2.
pd.options.mode.chained_assignment = None
new_Team = [team[:-2] if team[-2] == '-' else team for team in olympics.Team]
olympics.Team = new_Team

top_countries = olympics.groupby('Team').Team.count().sort_values(ascending = False)[0:40,]

In [ ]:
olympics = olympics.loc[olympics.Team.isin(top_countries.index)]

In [ ]:
top_sports = olympics.groupby('Sport').Sport.count().sort_values(ascending=False)[0:40,]


In [ ]:
olympics_top_sports = olympics.loc[olympics.Sport.isin(top_sports.index)]
olympics_our_sports = olympics[olympics.Sport.isin(['Swimming', 'Figure Skating', 'Table Tennis', 'Athletics'])]


In [ ]:
medals_top_sports = olympics_top_sports.loc[pd.notnull(olympics_top_sports.Medal)].groupby(['Team','Sport']).Team.count().unstack().fillna(0)
medals_our_sports = olympics_our_sports.loc[pd.notnull(olympics_our_sports.Medal)].groupby(['Team','Sport']).Team.count().unstack().fillna(0)

In [ ]:
mergings_top_sports = linkage(medals_top_sports, method = 'ward')
plt.figure(figsize=(5,10))
dendrogram(mergings_top_sports, labels = medals_top_sports.index, leaf_rotation = 0, leaf_font_size = 10, orientation = 'right')
plt.show()

# This dendrogram shows how closely certain countries are related based on their performance in the most popular olympic sports. 
# Many small European countries are clustered together, such as Slovenia, Estonia, Latvia, and Bulgaria.
# As we move down the dendrogram, we get to larger countries that have won more medals, such as Germany, Australia, and USA.

In [ ]:
mergings_our_sports = linkage(medals_our_sports, method = 'ward')
plt.figure(figsize=(5,10))
dendrogram(mergings_our_sports, labels = medals_our_sports.index, leaf_rotation = 0, leaf_font_size = 10, orientation = 'right')
plt.show()

# If we only consider our four sports, a lot of countries will be extremely close to each other, simply because
# they've won very few medals in these sports. 
# For example, Argentina is represented by the point (0, 0, 1, 0), which means they have won 0 medals in 
# athletics, figure skating, and table tennis, but 1 medal in swimming.
# Switzerland is represented by the point (0, 1, 0, 0), which means they have won 0 medals in athletics,
# swimming, and table tennis, but 1 medal in figure skating.
# So, the distance between Argentina and Switzerland is extremely small, because the values of their coordinates
# are almost all 0, meaning they will cluster together immediately.